Βαρδής Γεωργίλας - 3210027

# Β1

In [ ]:
!pip install --force-reinstall --no-cache-dir numpy==1.24.4 scipy==1.11.4 scikit-learn==1.3.2 gensim==4.3.2 fsspec==2023.6.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 157.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 154.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 155.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 183.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 169.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 175.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 189.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 276.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0

### Prepare dataset and metrics

In [ ]:
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd


volume_ds = load_dataset("AI-team-UoA/greek_legal_code", "volume")
chapter_ds = load_dataset("AI-team-UoA/greek_legal_code", "chapter")
subject_ds = load_dataset("AI-team-UoA/greek_legal_code", "subject")

results = []


def evaluate_and_store(y_true, y_pred, dataset_name, model_name, classes):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    results.append({
        'Dataset': dataset_name,
        'Model': model_name,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'Accuracy': accuracy
    })


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/9516 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/28536 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9511 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/9511 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/9516 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/28536 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9511 [00:00<?, ? examples/s]

### Ταξινόμηση με SVM με διαφορετικές αναπαραστάσεις του κειμένου

Ο σκοπός του αλγορίθμου είναι να βρίσκει το καλύτερο γραμμικό διαχωριστικό υπερεπίπεδο μεταξύ των κλάσεων, μεγιστοποιώντας την απόσταση ανάμεσα στα πιο κοντινά δείγματα διαφορετικών κλάσεων.

Εφαρμόζουμε το μοντέλο **Support Vector Machine (SVM)** για την κατηγοριοποίηση νομικών εγγράφων, χρησιμοποιώντας δύο διαφορετικές τεχνικές αναπαράστασης κειμένου:

- **Bag-of-Words (BoW)**: η κάθε λέξη αντιστοιχεί σε μια διάσταση και η τιμή της είναι η συχνότητά της στο κείμενο.
- **TF-IDF (Term Frequency - Inverse Document Frequency)**: Αναπαράσταση που μειώνει το βάρος των κοινών λέξεων και δίνει έμφαση σε λέξεις που είναι σημαντικές για το συγκεκριμένο έγγραφο.

Για κάθε αναπαράσταση:
- Χρησιμοποιούμε τον αλγόριθμο LinearSVC, γραμμικός ταξινομητής SVM.
- Πραγματοποιείται **αναζήτηση υπερπαραμέτρων**, ο αλγόριθμος δοκιμάζει όλους τους δυνατούς συνδυασμούς των καθορισμένων υπερπαραμέτρων και εκπαιδεύει το μοντέλο για κάθε συνδυασμό. Ο μέσος όρος των επιδόσεων αυτών χρησιμοποιείται για να επιλεγεί ο καλύτερος συνδυασμός υπερπαραμέτρων. Έτσι, αποφεύγεται το overfitting και βοηθά στην επιλογή του πιο γενικεύσιμου μοντέλου.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

results = []

for dataset_name, ds in zip(['volume', 'chapter', 'subject'], [volume_ds, chapter_ds, subject_ds]):

    X_train = ds['train']['text']
    y_train = ds['train']['label']
    X_val = ds['validation']['text']
    y_val = ds['validation']['label']
    X_test = ds['test']['text']
    y_test = ds['test']['label']

    for vectorizer_name in ['BoW', 'TF-IDF']:
        if vectorizer_name == 'BoW':
            vectorizer = CountVectorizer()
        else:
            vectorizer = TfidfVectorizer()

        # vectorize text data
        X_train_vec = vectorizer.fit_transform(X_train)
        X_val_vec = vectorizer.transform(X_val)
        X_test_vec = vectorizer.transform(X_test)

        # Hyperparameter tuning
        best_val_score = 0
        best_model = None
        best_params = {}

        hyperparameters = [
            {'C': 0.1, 'class_weight': None, 'loss': 'squared_hinge'},
            {'C': 1, 'class_weight': None, 'loss': 'squared_hinge'},
            {'C': 10, 'class_weight': None, 'loss': 'squared_hinge'},
            {'C': 0.1, 'class_weight': 'balanced', 'loss': 'squared_hinge'},
            {'C': 1, 'class_weight': 'balanced', 'loss': 'squared_hinge'},
            {'C': 10, 'class_weight': 'balanced', 'loss': 'squared_hinge'},
            {'C': 0.1, 'class_weight': None, 'loss': 'hinge'},
            {'C': 1, 'class_weight': None, 'loss': 'hinge'},
            {'C': 10, 'class_weight': None, 'loss': 'hinge'},
        ]


        for params in hyperparameters:

            model = OneVsRestClassifier(LinearSVC(
                C=params['C'],
                class_weight=params['class_weight'],
                loss=params['loss'],
                dual=True,
                random_state=42,
                max_iter=10000
            ))

            # τrain and evaluate
            model.fit(X_train_vec, y_train)
            val_score = model.score(X_val_vec, y_val)

            if val_score > best_val_score:
                best_val_score = val_score
                best_model = model
                best_params = params

        y_pred = best_model.predict(X_test_vec)
        evaluate_and_store(y_test, y_pred, dataset_name,f'SVM with {vectorizer_name}', sorted(set(y_test)))



### Logistic Regression με Word2Vec Embeddings


Το μοντέλο προβλέπει την κατηγορία ενός δείγματος με βάση ένα γραμμικό μοντέλο που μαθαίνει από τα διανύσματα λέξεων (Word2Vec). Τα διανύσματα των εγγράφων προκύπτουν από τον μέσο όρο των διανυσμάτων των λέξεων τους και κανονικοποιούνται με StandardScaler για καλύτερη απόδοση και ταχύτερη σύγκλιση.


- **Word2Vec Embeddings**: Κάθε λέξη μετατρέπεται σε ένα dense διάνυσμα 100 διαστάσεων, και κάθε κείμενο αναπαρίσταται ως ο μέσος όρος των διανυσμάτων των λέξεών του.
- Χρησιμοποιούμε το μοντέλο **LogisticRegression** εντός του OneVsRestClassifier με παραμέτρους C (ρυθμίζει το regularization) και class_weight (None ή balanced).
- Η επιλογή του καλύτερου μοντέλου γίνεται με δοκιμή διαφορετικών τιμών C και class_weight
- Η εκπαίδευση γίνεται σε κανονικοποιημένα διανύσματα από το training set και η αξιολόγηση στο validation set για να βρεθεί το καλύτερο μοντέλο




In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


combined_corpus = []

for split in ['train', 'validation', 'test']:
    combined_corpus.extend(
        [text.split() for text in volume_ds[split]['text']]  # Tokenize
    )

w2v_model = Word2Vec(
    sentences=combined_corpus,
    vector_size=100,
    window=5,
    min_count=2,
    workers=2
)


def create_doc_vectors(model, texts):
    return np.array([
        np.mean([model.wv[word] for word in text if word in model.wv]
                or [np.zeros(model.vector_size)], axis=0)
        for text in texts
    ])

# compute document vectors for all splits (same across datasets)
X_train_tok = [text.split() for text in volume_ds['train']['text']]
X_val_tok = [text.split() for text in volume_ds['validation']['text']]
X_test_tok = [text.split() for text in volume_ds['test']['text']]

X_train_vec = create_doc_vectors(w2v_model, X_train_tok)
X_val_vec = create_doc_vectors(w2v_model, X_val_tok)
X_test_vec = create_doc_vectors(w2v_model, X_test_tok)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_vec)
X_val_scaled = scaler.transform(X_val_vec)
X_test_scaled = scaler.transform(X_test_vec)


for dataset_name, ds in zip(['volume', 'chapter', 'subject'], [volume_ds, chapter_ds, subject_ds]):

    y_train = ds['train']['label']
    y_val = ds['validation']['label']
    y_test = ds['test']['label']


    best_score = 0
    best_model = None
    for C in [1, 10]:
        for class_weight in [None, 'balanced']:
            lr = OneVsRestClassifier(LogisticRegression(
                C=C,
                class_weight=class_weight,
                max_iter=1000,
                random_state=42
            ))
            lr.fit(X_train_scaled, y_train)
            val_score = lr.score(X_val_scaled, y_val)

            if val_score > best_score:
                best_score = val_score
                best_model = lr

    y_pred = best_model.predict(X_test_scaled)
    evaluate_and_store(y_test, y_pred, dataset_name,'LR with Word2Vec', sorted(set(y_test)))


### Random Forest με TF-IDF

Σε αυτό το μοντέλο εφαρμόζουμε τον αλγόριθμο **Random Forest** με την χρήση του OneVsRestClassifier, χρησιμοποιώντας ως χαρακτηριστικά τις αναπαραστάσεις TF-IDF των κειμένων. Ο OneVsRestClassifier εκπαιδεύει ένα μοντέλο Random Forest για κάθε ετικέτα ξεχωριστά, επιτρέποντας την ανεξάρτητη πρόβλεψη κάθε κατηγορίας. Ο Random Forest δημιουργεί πολλά δέντρα αποφάσεων και συνδυάζει τα αποτελέσματά τους για πιο σταθερές και ακριβείς προβλέψεις




In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier

for dataset_name, ds in zip(['volume', 'chapter', 'subject'], [volume_ds, chapter_ds, subject_ds]):

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train = vectorizer.fit_transform(ds['train']['text'])
    X_val = vectorizer.transform(ds['validation']['text'])
    X_test = vectorizer.transform(ds['test']['text'])

    y_train = ds['train']['label']
    y_val = ds['validation']['label']
    y_test = ds['test']['label']


    rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, max_depth=15, n_jobs=-1))
    rf.fit(X_train, y_train)
    val_score = rf.score(X_val, y_val)


    y_pred = best_model.predict(X_test)
    evaluate_and_store(y_test, y_pred, dataset_name,'Random Forest (TF-IDF)', sorted(set(y_test)))

## Evaluate

In [14]:
import pandas as pd

df_results = pd.DataFrame(results)
df_results['Index'] = df_results.groupby('Dataset').cumcount() + 1

for dataset, group in df_results.groupby('Dataset'):
    print(f"{dataset.capitalize()}:")
    print(group[['Index', 'Model','Precision', 'Recall', 'F1-score', 'Accuracy']].set_index('Index'))
    print()


Chapter:
                        Model  Precision  Recall  F1-score  Accuracy
Index                                                               
1                SVM with BoW     0.6547  0.6008    0.6105    0.7181
2             SVM with TF-IDF     0.7782  0.7326    0.7418    0.8116
3            LR with Word2Vec     0.3137  0.2630    0.2654    0.3823
4      Random Forest (TF-IDF)     0.4740  0.4055    0.4253    0.5832

Subject:
                        Model  Precision  Recall  F1-score  Accuracy
Index                                                               
1                SVM with BoW     0.4482  0.4149    0.4077    0.6042
2             SVM with TF-IDF     0.5728  0.5612    0.5448    0.7252
3            LR with Word2Vec     0.2070  0.1809    0.1769    0.3134
4      Random Forest (TF-IDF)     0.2608  0.2294    0.2245    0.4334

Volume:
                        Model  Precision  Recall  F1-score  Accuracy
Index                                                               
1     

### Συμπεράσματα

- Τα αποτελέσματα δείχνουν ότι το μοντέλο SVM with TF-IDF υπερέχει σταθερά και στις τρεις κατηγορίες, επιτυγχάνοντας τις υψηλότερες τιμές σε όλες τις μετρικές. Το SVM with BoW ακολουθεί με ελαφρώς χαμηλότερες επιδόσεις, παραμένοντας ωστόσο αξιόπιστο, ενώ το Random Forest with TF-IDF παρουσιάζει μέτρια αποτελέσματα, καλύτερα από το LR with Word2Vec αλλά υποδεέστερα των SVM μοντέλων. Το LR with Word2Vec εμφανίζει τις χαμηλότερες επιδόσεις σε όλες τις κατηγορίες υποδεικνύοντας ότι ίσως δεν είναι κατάλληλο για αυτές τις εφαρμογές. Συνολικά, το SVM with TF-IDF αναδεικνύεται ως η πιο αξιόπιστη επιλογή για το συγκεκριμένο task.

- Η ετικέτα "συλλογή" (volume) παρουσιάζει την υψηλότερη απόδοση, γεγονός που σχετίζεται με τον πιο γενικό και σαφώς διακριτό χαρακτήρα της, καθώς και με την ύπαρξη χαρακτηριστικών λέξεων στο κείμενο και περιορισμένου αριθμού κατηγοριών με καλύτερη κατανομή. Αντίθετα, οι ετικέτες "κεφάλαιο" και ιδιαίτερα "θέμα" είναι πιο εξειδικευμένες, περιλαμβάνουν περισσότερες και λιγότερο συχνές κατηγορίες, και συχνά βασίζονται σε πιο έμμεσα ή ασαφή στοιχεία του κειμένου, κάνοντας την ταξινόμηση πιο απαιτητική.
